In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Load the dataset
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Explore the data
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)
print("Missing values in train data:\n", train_data.isnull().sum().sort_values(ascending=False))

# Drop irrelevant features (e.g., 'Id')
train_data.drop(columns=['Id'], inplace=True)
test_data_ids = test_data['Id']  # Save test data IDs for final submission
test_data.drop(columns=['Id'], inplace=True)

# Fill missing values
# For numerical columns, use the median
train_data.fillna(train_data.median(), inplace=True)
test_data.fillna(test_data.median(), inplace=True)

# One-hot encode categorical features
train_data = pd.get_dummies(train_data, drop_first=True)
test_data = pd.get_dummies(test_data, drop_first=True)

# Align train and test data to have the same features, excluding SalePrice
train_data, test_data = train_data.align(test_data, join='left', axis=1)

# Ensure the target variable 'SalePrice' is not part of the test data
test_data = test_data.drop(columns=['SalePrice'], errors='ignore')

# Separate features (X) and target variable (y)
X = train_data.drop(columns=['SalePrice'])
y = train_data['SalePrice']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
test_data = scaler.transform(test_data)

# Train a Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Train a Decision Tree Regressor
dt_model = DecisionTreeRegressor(max_depth=5, random_state=42)
dt_model.fit(X_train, y_train)

# Evaluate the models
def evaluate_model(model, X, y_true, model_name):
    y_pred = model.predict(X)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} - MSE: {mse:.2f}, R²: {r2:.2f}")
    return y_pred

print("\nModel Evaluation on Validation Set:")
lr_predictions = evaluate_model(lr_model, X_val, y_val, "Linear Regression")
dt_predictions = evaluate_model(dt_model, X_val, y_val, "Decision Tree")

# Use the better model (Decision Tree in this case) to predict house prices for the test set
final_predictions = dt_model.predict(test_data)

# Save the predictions to a CSV file
submission = pd.DataFrame({'Id': test_data_ids, 'SalePrice': final_predictions})
submission.to_csv('submission.csv', index=False)

print("\nPredictions saved to 'submission.csv'")


Train data shape: (10, 8)
Test data shape: (10, 7)
Missing values in train data:
 Id              0
LotArea         0
OverallQual     0
YearBuilt       0
GrLivArea       0
FullBath        0
BedroomAbvGr    0
SalePrice       0
dtype: int64

Model Evaluation on Validation Set:
Linear Regression - MSE: 1280734363.59, R²: -0.92
Decision Tree - MSE: 256930000.00, R²: 0.61

Predictions saved to 'submission.csv'
